In [1]:
import json

from jinja2 import Template, StrictUndefined
from jinja2.exceptions import TemplateError

import yaml

In [2]:
# Planning Agent
with open('prompts/planning_agent.yaml', 'r') as file:
    planning_agent_prompt = yaml.load(file, Loader=yaml.FullLoader)

user_template = Template(
    planning_agent_prompt['user'],
    undefined=StrictUndefined
)

contents = {
    "trope": "some trope",
    "setting": "some setting",
    "characters": [
        {
            "uid": "uid1",
            "description": "test"
        },
        {
            "uid": "uid2",
            "description": "test"
        },
    ],
    "history": [
        "action1",
        "action2",
    ]
}
filled = user_template.render(**contents)
print(filled)

[Scene Description]
* trope: some trope
* setting: some setting

[Characters]

uid: uid1
* description: test

uid: uid2
* description: test


[History]
* action1
* action2



In [3]:
# Emotion Predictor
with open('prompts/emotion_predictor.yaml', 'r') as file:
    emotion_predictor_prompt = yaml.load(file, Loader=yaml.FullLoader)

user_template = Template(
    emotion_predictor_prompt['user'],
    undefined=StrictUndefined
)
contents = {
    "source": "some source",
    "character": "some character"
}

filled = user_template.render(**contents)
print(filled)

Source: some source
Character: some character


In [8]:
# Emotion Predictor
with open('prompts/action_predictor.yaml', 'r') as file:
    action_predictor_prompt = yaml.load(file, Loader=yaml.FullLoader)

system_template = Template(
    action_predictor_prompt['system'],
    undefined=StrictUndefined
)
user_template = Template(
    action_predictor_prompt['user'],
    undefined=StrictUndefined
)

with open("assets/sims_interactions.json", "r") as f:
    actions = json.load(f)
    
system_contents = {
    "actions": actions
}

system_filled = system_template.render(**system_contents)
print(len(system_filled))
print(system_filled)

17752
You are a creative storyteller that does the following.
Given informations about a character (Character) and context of a story scene (Scene Context)
(1) write the next action that the character will perform in the scene.
(2) update the character state after performing this action

# Given Informations
## Character
description: character persona that you will be taking
state:
* polarity: sentiment towards the character (positive/neutral/negative)
* emotion: emotional state of the character
* social_relations: a one-directional relationship with other characters. (ex. what character1 thinks of character2)
    * character_uid: uid of the target character
    * emotion: emotional relation towards the other character
    * knowledge: what the character knows about the other character 

## Scene Context
description of the current state of the scene

# Action Prediction
## Definition of 'Action'
a character performs an 'action' through the following steps
* 'think': consider what the c